## LangChain 사용해서 지원사업 공고 파일(pdf, hwp, png)을 요약해보자!

In [3]:
from openai import OpenAI
from getpass import getpass
from tqdm import tqdm

from utils.get_text import get_hwp_text, get_pdf_text # pdf, hwp에서 텍스트를 추출하는 사용자 지정 함수 

- LangChain 모델이 참고할 비슷한 용어 사전
- 개발팀이 직접 정의
- (추후에 유사한 단어가 나오면 사전에 추가하는 기능도 구현할 예정)

In [4]:
# 정의한 비슷한 용어 사전
keyword_dictionary = {
                        '신청기간' : ['신청 기간', '접수 기간'],
                        '신청방법' : ['지원 대상', '사업 대상', '융자 대상'],
                        '지원제외' : ['지원 제외','지원 취소', '제외 대상', '제한 대상'],
                        '유의사항' : ['의무사항', '지원 취소 및 제외', '기타 안내사항'],
                        '제출 서류' : ['필요 서류', '서류 제출 목록'],
                        '선정 절차' : ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '평가 방향', '선정 기준'],
                        '지원 내용' : ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용'],
                        '문의처' : ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의'],
                        '기타' : ['기타']
                        }

#### model 구현 준비
- OpenAI 객체 생성
- 파일 텍스트 추출
- model이 참고할 기본 템플릿 작성

In [5]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [7]:
# 파일 경로 설정 (백슬래시 문제 해결을 위해 r"" 사용)
file_path = f"금융/[경남] 진주시 2025년 중소기업 개별 수출보험료 지원사업 공고.hwp"

# 지원사업명 저장 (지원경로를 분할하여 사용)
title = file_path.split('/')[1].split('.')[0]

# 파일 타입 저장 (지원경로를 분할하여 사용)
file_type = file_path.split('/')[1].split('.')[1]

# file_type이 pdf면 get_pdf_text, hwp면 get_hwp_text를 사용하여 텍스트 추출
if file_type == 'pdf':
    full_text = get_pdf_text(file_path)
elif file_type == 'hwp':
    full_text = get_hwp_text(file_path)

In [8]:
base_template = {f'''
📌 **[{title}] 요약 안내**

📅 **신청기간**  

📝 **신청방법**  

🚫 **지원제외 대상**   

⚠ **유의사항**  

📑 **제출서류**  

📌 **선정절차**  

💰 **지원내용**  

📞 **문의처**  

📢 **기타 안내**  
'''
}

#### LangChain 모델로 간단하게 요약해보기
- (고도화 예정)

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
당신은 파일 요약 전문가입니다. 사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 요약해야합니다.
파일에 없는 내용은 공백으로 남겨두십시오.
요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니, 사전에 해당하는 단어가 나오면, 참고하여 요약해주십시오.

제공하는 템플릿 양식 : {base_template}
단어 정의 사전 : {keyword_dictionary}

---------------------------------------------------
사용자 요구 파일 : {text}

"""

# 파일 요약을 위한 프롬프트 템플릿을 정의
prompt = PromptTemplate.from_template(template)

# OpenAI의 GPT-4o 모델을 사용하여 채팅 모델을 생성
model = ChatOpenAI(
    model='gpt-4o',  # 사용할 언어 모델 지정
    api_key=MY_API_KEY  # OpenAI API 키 입력
)

# 모델의 출력을 문자열로 변환하는 출력 파서
output_parser = StrOutputParser()

# 체인(Chain) 구성: 프롬프트 → 모델 → 출력 파서 순서로 연결
chain = prompt | model | output_parser

In [10]:
print(chain.invoke({'base_template':base_template,
              'keyword_dictionary':keyword_dictionary,
              'text':full_text}))

📌 **[[경남] 진주시 2025년 중소기업 개별 수출보험료 지원사업 공고] 요약 안내**

📅 **신청기간**  
공고일로부터 2025년 12월 15일까지(또는 사업비 소진 시까지)

📝 **신청방법**  
보험심사 담당자와 협의 후 Fax 또는 E-메일 접수  
방문: (우)51430 창원시 의창구 중앙대로 257 경남무역회관 5층  
E-메일 : lmh00969@ksure.or.kr  
Fax : (02)6234-1443

🚫 **지원제외 대상**  

⚠ **유의사항**  
신청 내용이 허위로 밝혀질 경우 지원금 환급 및 향후 지원 중단  
기타 접수기관에서 지원신청 심사 등과 관련 요청 시, 관련 자료 제출 필요

📑 **제출서류**  
수출보험료 지원 신청서 1부, 사업자등록증 1부

📌 **선정절차**  
수출보험 가입 희망업체가 지원신청서 및 사업자등록증을 한국무역보험공사 경남지사에 제출하면 지원자격 심사 선정

💰 **지원내용**  
진주시에 본사 또는 지사(공장) 소재한 중소기업, 업체당 2,000천원 이내  
지원보험: 수출보험, 신용보증, 수입보험, 환변동보험 등

📞 **문의처**  
한국무역보험공사 경남지역본부 (055)286-9391  
진주시청 기업통상과 (055)749-8146

📢 **기타 안내**  
